In [5]:
library(rhdf5)
library(Matrix)
library(SingleCellExperiment)

In [2]:
mydir = "/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/"

In [60]:
# Day 30
file1 = paste0(mydir,"pool1_13_noddd_D30/pool1_13_noddd_D30.scanpy.w_metadata.w_celltype.scanpy.h5")

In [4]:
# Day 52
file2 = paste0(mydir,"pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.h5")

In [61]:
filename = file1
# filename = file2

In [62]:
cbind(h5ls(filename)$group,
h5ls(filename)$name)

/,X
/X,data
/X,indices
/X,indptr
/,obs
/,obsm
/,uns
/uns,batch_categories
/uns,celltype_categories
/uns,donor_id_categories
/uns,pool_id_categories


In [11]:
# gene_anno_df = data.frame(gene_name = h5read(file = filename, "var")$index,
#                          ensembl_gene_id = h5read(file = filename, "var")[['gene_ids-0']])
head(gene_anno_df)

gene_name,ensembl_gene_id
MIR1302-10,ENSG00000243485
FAM138A,ENSG00000237613
OR4F5,ENSG00000186092
RP11-34P13.7,ENSG00000238009
RP11-34P13.8,ENSG00000239945
AL627309.1,ENSG00000237683


In [64]:
ind = h5read(file = filename, "obs")$index # cell names
# var = h5read(file = filename, "var")$index # gene names
var = h5read(file = filename, "var")[['gene_ids-0']] # ensembl gene ids 

In [65]:
head(ind)
head(var)

[1] "AAACCTGAGAGCTATA-1-0" "AAACCTGAGCTAGTCT-1-0" "AAACCTGAGTGTTAGA-1-0"
[4] "AAACCTGCAACCGCCA-1-0" "AAACCTGCAAGGGTCA-1-0" "AAACCTGCAATCCGAT-1-0"

[1] "ENSG00000243485" "ENSG00000237613" "ENSG00000186092" "ENSG00000238009"
[5] "ENSG00000239945" "ENSG00000237683"

In [66]:
X = h5read(file = filename, "X")
X_data = as.integer(X$data)
X_indptr = as.integer(X$indptr)
X_indices = as.integer(X$indices)
rm(X)

In [67]:
sMat = sparseMatrix(
i = X_indices,
p = X_indptr,
x = X_data,
dims = c(length(var), length(ind)),
index1 = FALSE
)
rownames(sMat) <- as.character(var)
colnames(sMat) <- as.character(ind)

In [68]:
rownames(sMat)[grep("SOX",rownames(sMat))]

character(0)

In [69]:
head(rownames(sMat))

[1] "ENSG00000243485" "ENSG00000237613" "ENSG00000186092" "ENSG00000238009"
[5] "ENSG00000239945" "ENSG00000237683"

In [20]:
# rownames(sMat) <- gene_anno_df$ensembl_gene_id

In [70]:
cData_raw = h5read(file = filename, "obs")

In [71]:
# now the column data information
colinfo_all = apply(cbind(h5ls(filename)$group,
    h5ls(filename)$name)
    ,1,paste0, collapse = "/")
colinfo_all

[1] "//X"                        "/X/data"                   
 [3] "/X/indices"                 "/X/indptr"                 
 [5] "//obs"                      "//obsm"                    
 [7] "//uns"                      "/uns/batch_categories"     
 [9] "/uns/celltype_categories"   "/uns/donor_id_categories"  
[11] "/uns/pool_id_categories"    "/uns/sample_id_categories" 
[13] "/uns/time_point_categories" "/uns/treatment_categories" 
[15] "//var"

In [72]:
colinfo <- grep("categories",colinfo_all, value = TRUE)
colinfo

[1] "/uns/batch_categories"      "/uns/celltype_categories"  
[3] "/uns/donor_id_categories"   "/uns/pool_id_categories"   
[5] "/uns/sample_id_categories"  "/uns/time_point_categories"
[7] "/uns/treatment_categories"

In [73]:
for (col in colinfo){
    print(col)
    cat = gsub("/uns/","",gsub("_categories","",col))
    cats = h5read(file = filename, col)
#     print(cats)
    #print(head(cData_raw[[cat]]))
    cData_raw[[cat]] = cats[cData_raw[[cat]]+1]
}

[1] "/uns/batch_categories"
[1] "/uns/celltype_categories"
[1] "/uns/donor_id_categories"
[1] "/uns/pool_id_categories"
[1] "/uns/sample_id_categories"
[1] "/uns/time_point_categories"
[1] "/uns/treatment_categories"


In [74]:
head(cData_raw)

index,batch,donor_id,sample_id,cluster_id,time_point,pool_id,treatment,n_counts_all,celltype
AAACCTGAGAGCTATA-1-0,0,HPSI0714i-iudw_1,5245STDY7387187,0,D30,pool1,NONE,4370,DA
AAACCTGAGCTAGTCT-1-0,0,HPSI0114i-eipl_1,5245STDY7387187,3,D30,pool1,NONE,19176,Epen1
AAACCTGAGTGTTAGA-1-0,0,HPSI0614i-liqa_1,5245STDY7387187,2,D30,pool1,NONE,11318,Sert
AAACCTGCAACCGCCA-1-0,0,HPSI0115i-paim_1,5245STDY7387187,1,D30,pool1,NONE,5091,Astro
AAACCTGCAAGGGTCA-1-0,0,HPSI0114i-eipl_1,5245STDY7387187,1,D30,pool1,NONE,8612,Astro
AAACCTGCAATCCGAT-1-0,0,HPSI0115i-paim_1,5245STDY7387187,1,D30,pool1,NONE,6807,Astro


In [75]:
cData <- as.data.frame(cData_raw)

In [76]:
# build the sce object
sce <- SingleCellExperiment(
assays = list(counts = sMat),
colData = cData)
sce

class: SingleCellExperiment 
dim: 32738 250923 
metadata(0):
assays(1): counts
rownames(32738): ENSG00000243485 ENSG00000237613 ... ENSG00000215616
  ENSG00000215611
rowData names(0):
colnames(250923): AAACCTGAGAGCTATA-1-0 AAACCTGAGCTAGTCT-1-0 ...
  TTTGTCATCCCTTGTG-1-33 TTTGTCATCCGTAGTA-1-33
colData names(10): index batch ... n_counts_all celltype
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [77]:
# sce_neurons = sce[,sce$celltype %in% c("DA","Sert") & sce$treatment == 'ROT']
sce_neurons = sce[,sce$celltype %in% c("DA","Sert")]
sce_neurons = sce_neurons[,sample(colnames(sce_neurons),10000)]

In [78]:
sce_neurons

class: SingleCellExperiment 
dim: 32738 10000 
metadata(0):
assays(1): counts
rownames(32738): ENSG00000243485 ENSG00000237613 ... ENSG00000215616
  ENSG00000215611
rowData names(0):
colnames(10000): GTCGGGTCATGTCGAT-1-19 ACCAGTATCAAGAAGT-1-33 ...
  GATTCAGCAGCATACT-1-19 TGTATTCCATCGATTG-1-1
colData names(10): index batch ... n_counts_all celltype
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [79]:
saveRDS(sce_neurons,"/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/sce_neurons_d30_subsample_10000cells.Rds")
# saveRDS(sce_neurons,"/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/sce_neurons_d52_none_subsample_10000cells.Rds")
# saveRDS(sce_neurons,"/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/sce_neurons_d52_rot_subsample_10000cells.Rds")

In [80]:
load("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/neuMat/mean_markers.rda")
load("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/neuMat/coefs.rda")
load("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/neuMat/modules_info.rda")
load("/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/neuMat/sd_markers.rda")

In [81]:
dataStandardize <- function(fpkm, standardize=TRUE){
    num.detected <- sum(rownames(fpkm) %in% names(mean_markers))
    if(num.detected < length(mean_markers)/1.5){
        stop(paste0("Too few marker genes. Only ", num.detected, " marker genes are available while at least ", ceiling(length(mean_markers)/1.5), " is required."))
    }
    if(standardize){
        fpkm <- log10(fpkm + 1)
    }
    
    fpkm.filled <- t(sapply(names(mean_markers), function(gene){
        if(sum(rownames(fpkm)==gene)>0){
            return(as.numeric(fpkm[gene,]))
        } else{
            return(rnorm(ncol(fpkm), mean_markers[gene], sd_markers[gene]))
        }
    }))
    if(standardize){
        fpkm.std <- apply(fpkm.filled, 2, function(e){ (e - mean_markers[rownames(fpkm.filled)]) / sd_markers[rownames(fpkm.filled)] })
    } else{
        fpkm.std <- fpkm.filled
    }
    colnames(fpkm.std) <- colnames(fpkm)
    return(fpkm.std)
}

In [82]:
predictNMI <- function(fpkm, standardize = TRUE, returnFpkmStd = FALSE){
    require("Matrix")
    fpkm <- dataStandardize(fpkm, standardize = standardize)
    fpkm.std <- rbind(rep(1, ncol(fpkm)), fpkm)
    pred <- t(fpkm.std) %*% coefs
    pred <- exp(pred) / (exp(pred) + 1)
    
    pred.overall <- as.numeric(pred %*% as.matrix(modules_info$weight)) / sum(modules_info$weight)
    pred.discriminable <- as.numeric(pred[,modules_info$discriminable] %*% as.matrix(modules_info$weight[modules_info$discriminable])) / sum(modules_info$weight[modules_info$discriminable])
    pred.activity <- as.numeric(pred[,modules_info$discriminable & modules_info$matureHigh] %*% as.matrix(modules_info$weight[modules_info$discriminable & modules_info$matureHigh])) / sum(modules_info$weight[modules_info$discriminable & modules_info$matureHigh])
    pred.integrated <- data.frame(overall = pred.overall, discriminable = pred.discriminable, activity = pred.activity)
    rownames(pred.integrated) <- colnames(fpkm)
    
    if(returnFpkmStd){
        res <- list(modularNMI = pred, overallNMI = pred.integrated, fpkmStd = fpkm.std)
    } else{
        res <- list(modularNMI = pred, overallNMI = pred.integrated)
    }
    return(res)
}

In [83]:
nmi = predictNMI(counts(sce_neurons))

In [84]:
str(nmi)

List of 2
 $ modularNMI:Formal class 'dgeMatrix' [package "Matrix"] with 4 slots
  .. ..@ x       : num [1:1090000] 0.59 0.59 0.59 0.59 0.59 ...
  .. ..@ Dim     : int [1:2] 10000 109
  .. ..@ Dimnames:List of 2
  .. .. ..$ : chr [1:10000] "GTCGGGTCATGTCGAT-1-19" "ACCAGTATCAAGAAGT-1-33" "TGGCCAGGTCTCTTTA-1-15" "CTCACACCAGATAATG-1-26" ...
  .. .. ..$ : NULL
  .. ..@ factors : list()
 $ overallNMI:'data.frame':	10000 obs. of  3 variables:
  ..$ overall      : num [1:10000] 0.473 0.482 0.484 0.482 0.493 ...
  ..$ discriminable: num [1:10000] 0.444 0.456 0.469 0.433 0.476 ...
  ..$ activity     : num [1:10000] 0.307 0.355 0.3 0.291 0.311 ...


In [85]:
nmi$overallNMI$celltype = sce_neurons$celltype
nmi$overallNMI$treatment = sce_neurons$treatment
# nmi$overallNMI$time_point = sce_neurons$time_point

In [86]:
head(nmi$overallNMI)

,overall,discriminable,activity,celltype,treatment
GTCGGGTCATGTCGAT-1-19,0.4728026,0.4439646,0.3068475,Sert,NONE
ACCAGTATCAAGAAGT-1-33,0.4822061,0.4555255,0.3546429,DA,NONE
TGGCCAGGTCTCTTTA-1-15,0.4841568,0.4690050,0.3002614,Sert,NONE
CTCACACCAGATAATG-1-26,0.4822049,0.4329310,0.2910879,Sert,NONE
CCCTCCTGTCAAGCGA-1-6,0.4931809,0.4764145,0.3114766,Sert,NONE
CGATGGCAGTGGTAGC-1-20,0.4893730,0.4609344,0.3057609,Sert,NONE


In [59]:
# nmi_d52_rot = nmi
# saveRDS(nmi_d52_rot, "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/neuMat/nmi_neurons_day52_rot.Rds")

In [45]:
# nmi_d52 = nmi
# saveRDS(nmi_d52, "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/neuMat/nmi_neurons_day52_none.Rds")

In [87]:
nmi_d30 = nmi
saveRDS(nmi_d30, "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/neuMat/nmi_neurons_day30.Rds")